# 树

In [3]:
def test():
    print('1')
    print('3')
    return 'r'
    print('2')
    
test()

1
3


'r'

## 定义问题

当我们说“树形结构”的时候，那是从自然界的大树上得到的启发，从根开始不断展开的枝丫，代表了从单一到复杂的一层层展开，在现实世界充满了这样的事物：国家的行政区划（以及任何组织的架构）、电商网站的商品分类、论坛里的板块等等。

<img src="assets/tree-graph.png" width="700">

> 虽然自然界里树是根在下面而向上分支，但是我们在处理树形结构时，如图这样的从上向下分支更容易画也更容易看。

解决问题的第一步是先要清晰明确地定义问题，我们来尝试把我们对树的直觉印象书写成尽可能清晰明确的定义和表述。在一个树形结构里：
* 树（*tree*）由节点（*node*）和连接节点的边（*edge*）组成；
* 总有一个节点是分支的起点，它分出了所有其他的节点，这个节点叫根节点（*root node*）；
* 一个节点分支出来的节点叫它的“子节点（*child nodes*）”，它是其子节点的“父节点（*parent node*）”；上图中节点 A 的父节点是根节点，子节点是 B 和 C，而节点 C 的父节点是 A，子节点是 D 和 E；节点 D 的父节点是 C，而它没有子节点；
* 拥有共同父节点的两个节点互为“兄弟姐妹（*sibling nodes*）”；比如上图中 B 和 C，D 和 E；
* 没有子节点的节点也叫“叶子节点（*leaf node*）”；比如上图中的 D 和 E；
* 一个节点的父节点，以及父节点的父节点…直至根节点，都是这个节点的“祖先节点（*ancestor nodes*）”；比如上图中 E 的祖先节点包括 C、A 和 根节点；
* 一个节点的所有子节点，以及所有子节点的子节点…直至叶子节点，都是这个节点的“后代节点（*descendant nodes*）”；
* 一个节点 X 和它所有后代节点、以及这些节点之间的边，也组成一棵树，叫做原数的一棵（以 X 为根的）“子树（*sub-tree*）”；
* 根节点没有父节点也没有祖先节点；
* 除了根节点以外的任何节点，有且只有 1 个父节点，有至少 1 个祖先节点；
* 任何节点都可以有 0、1 或者多个子节点，可以有  0、1 或者多个后代节点；
* 一个节点和它的任一祖先或者后代节点之间，一定存在一条由边收尾连接组成的路径（*path*），比如上图中根节点是 E 的祖先，它们之间的路径是 `root -> A -> C -> E`；这个路径就像是两个节点之间的亲属关系链；
* 两个节点之间 *path* 的长度，就是它由几条边组成，决定了这两个节点之间隔了多少“代”，也叫节点之间的“距离（*distance*）”；
* 一个节点和根节点之间的 *distance* 经常有特别的含义，相当于该节点在树的“第几层级（*tier*）”；比如在行政区划里哪些是一级节点（省、直辖市、自治区）哪些是二级节点等；
* 有些树里节点的子节点是有顺序的，叫做“有序树（*ordered tree*）”；如果我们不特别指出，那就是不考虑这种顺序概念。

上面这种“把直觉规则化”的过程，对后面设计解决方案是至关重要的，大家可以自己尝试，多多体会。

> 事实上在数学图论里有对树的[更严谨更数学化的定义](https://en.wikipedia.org/wiki/Tree_(graph_theory))，不过上面的表述对目前的我们来说基本够用了。

## 分析操作场景

可以看到树和我们前面讲的所有数据结构都不一样，没办法用一种“线性（*linear*）”结构来表达，如何在计算机中实现一个树形数据结构？面对这样的问题，我们应该如何思考呢？

一个优秀的起点是问题解决的一半。设计数据结构的起点是：**思考我们会怎么操作和使用这个数据结构**。

假定我们已经有个一个树形数据结构，我们可能会有这些操作场景（可以尝试用行政区划或者论坛版块等熟悉的实例来帮助思考）：
* 查找和遍历：
  * 给定一个 *node* 找出它的 *parent* ✪
  * 给定一个 *node* 找出它所有的 *children* ✪
  * 给定一个 *node* 找出它某个特定 *child*
  * 给定一个 *node* 找出它所有的 *siblings*
  * 给定一个 *node* 遍历其 *sub-tree* 即找出它的所有 *descendants* ✪
  * 给定 *node* *A* 和 *B*，找到 *A* 和 *B* 之间的 *path* 和 *distance*
  * 给定一个 *node* 确定它的 *tier*
* 编辑
  * 给一个 *node* 增加一个 *child* ✪
  * 删除一个 *node* ✪ -> 新一步思考：这意味着什么？删除整个子树还是？
  * 修改一个 *node* 的 *parent* ✪

差不多就这些，其中标记了 ✪ 号的是感觉特别重要和基础的操作。

通过形象化的图示、对基本概念的定义和表述、对可能操作场景的罗列，已经增加了很多我们对问题理解的深度和全面度，下来我们可以尝试做出一些初步设计判断了。

## 设计初步方案

从前面的分析，我们可以有这么一些收获：
* 树中节点之间的父子关系是核心和本质的内容；
* 表达这种父子关系的关键是节点，节点应保存父节点和子节点相关信息；
* 树的结构上带有显著的“递归”特点，可以有助于我们的设计。

如果你不记得“递归（*recursion*，*recursive*）”是怎么回事了，可以温习[递归函数](p2-4-recursion.ipynb)一章。

树也是递归的一个典型例子，因为一棵树可以看做由**根节点、根节点的子节点和以这些子节点为根的子树**合起来组成的，如果我们想实现对树进行操作的函数 `f()`，那么我们可以让这个函数接受一个树的根节点作为输入参数，这个函数大致上会是这个样子：

```python
def f(root):
    # 对 root 做一些操作
    # 然后取出 root 的所有子节点，对其中每个子节点调用 f() 本身
    for node in root.children:
        f(node)
```

也就是说，我们只要对某个节点做操作，然后获取这个节点的所有子节点，递归调用自己，就能对整个树做操作了。

在树形数据结构中，“获取一个节点的所有子节点”是至关重要的操作。

据此我们可以做出这么一个初步的设计：

In [ ]:
# 请在这里输入代码
class TreeNode:
    def __init__(self, name='root', data=None, parent=None, chilren=None):
        self.name = name
        self.data = data
        
        if parent:
            # 确认 parent 参数是 TreeNode 类型
            assert isinstance(parent, TreeNode)
            parent.add_child(self)
        self.patent = parent
        
        self.children = []
        if children:
            for chil in children:
                self.add_child(child)
                
    def add_child(self, node):
        # 1. 确认 node 参数是 TreeNode 类型
        # 2. 将要加入的子节点的 parent 属性设为自己
        # 3. 然后将其加入 children 列表
        assert isinstance(node, TreeNode)
        node.parent = self
        self.children.append(node)

In [ ]:
# 不时比对的结果。
class TreeNode:
    def __init__(self, name='root', data=None, parent=None, children=None):
        self.name = name
        self.data = data
        
        if parent:
            assert isinstance(parent, TreeNode)
            parent.add_child(self)
        self.parent = parent
        
        self.children = []
        if children:
            for child in children:
                self.add_child(child)
                
    def add_child(self, node):
        assert isinstance(node, TreeNode)
        node.parent = self
        self.children.append(node)

In [100]:
def test2():
    l = []
    for i in l:
        return 1
    else:
        return 2
        
print(test2())

2


In [89]:
# 自己理解后，进行总结后，自己单纯自己写的版本：
class TreeNode:
    def __init__(self, name='root', data=None, parent=None, children=None):
        self.name = name
        self.data = data
        
        if parent:  # 这里是根节点的特点
            assert isinstance(parent, TreeNode)
            parent.add_child(self)
        self.parent = parent  # 这个是第二次没有想到的
            
        self.children = []
        if children:  # 这里是最后一片叶子的特点。头、尾是有单独的特点的。
            for child in children:
                assert isinstance(child, TreeNode)  # 这个是我结合上面 parent 的特点，添加上去的。
                self.add_child(child)
                
#         self.children = []
#         这个是第二次自己尝试的版本
#         for child in children:
#             assert isinstance(child, TreeNode)
#             self.add_child(child)
    
    def add_child(self, node):
        assert isinstance(node, TreeNode)
        node.parent = self
        self.children.append(node)
    
    def add_children(self, nodes):
        for node in nodes:
            assert isinstance(node, TreeNode)
        for node in nodes:
            node.parent = self
            self.children.append(node)
    
    def remove(self, node):
        node.parent = None
        self.children.remove(node)
    
    def move(self, node, new_parent):
        node.parent.remove(node)
        node.parent = new_parent
        new_parent.add_child(node)

#     # Version 1
#     def find(self, name, notfind = True):
#         notfindbool = notfind
#         for child in self.children:
#             if child.name == name:
#                 print(child.data['tag'])
#                 notfindbool = False
#             child.find(name, notfindbool)
#         if notfindbool:
#             print('not find')
    
    # Version 2
    def find(self, name):
        if self.name == name:
            return self
        
        for child in self.children:
#             if child.name == name: # version 1
#                 return child
#             child.find(name)
#             if child.find(name):  # version 2  # 相当于在 if 里面一直寻找，递归寻找之后得到正确，就回到这一层，没有传递下去。
#                 return child
            t = child.find(name)
            if t:
                return t
        else:
            return None
        
# 总结一下：
# 递归的基本形式就是：

# def func(self):
#     做递归的元操作
#     for item in items:
#         item.func()
        
# 2020年2月20日 22:28:51

    def get_siblings(self, name):
        thisNode = self.find(name)
        thisParent = thisNode.parent
        siblings = []
        for child in thisParent.children:
            if child.name != name:
                siblings.append(child)
        return siblings
        
    def get_descendants(self, name):
        node = self.find(name)
        node.print_tree()
        
#         # version 2
#         node = self.find(name)
#         descendants = []
        
#         for child in node.children:
#             descendants.append(child)
#             descendants.append(child.get_descendants(child.name))
            
#         return descendants
    
#         思考：
#         如果是 root，没有区别；
#         如果是最后一个，需要返回 None
#         如果是中间的，那么和 root 没有区别。
#         找到 child，找到 child 的 child，一直下去，直到是最后一个。
        
        
#         # version 1
#         descendants = []
#         this_node = self.find(name)
        
#         for child in this_node.children:
#             descendants.append(child)
#             if child.children:
#                 child.get_descendants(name)
#         else:
#             return descendants
    
    def get_tier(self, name, tier=0):
        if self.name == name:
            return tier
        
        for child in self.children:
            t = child.get_tier(name, tier+1)
            if t:
                return t  # 把 tier+1 改成 t，前面都对，但是 最后一层会返回 None，why？
        else:
            return None # 最后一层报错，是因为这里写的是 ‘not exit’，然后就真的在最后一层判断的时候，返回这个了。。。为什么改成 None 就好了呢？
        
#         # version 1
#         if self.children:
#             for child in self.children:
#                 if child.name == name:
#                     return tier+1
#                 child.get_tier(name, tier+1)
    
    # Version 4
    # 虽然一次性把课程的范例修改了过来，感觉很神奇，自己能够把不同位置的 code 进行转化
    # 更加没有理清楚其中的逻辑。
    # 冥冥中感觉关键是之前逻辑的考虑：开头 和 结尾 是不同的形式，所以要单独考虑。也就是在 print_tree 里面进行了区别对待 self 和 它的 children
    # 在 _print_tree 里面区别对待了 children 里面的最后一个。
    def _print_tree(self, _prefix='', _last=True):
        print(''.join([_prefix, '┗━━' if _last else '┣━━', self.name]))
        _prefix += '   ' if _last else '┃  '
        count = len(self.children)
        for n, node in enumerate(self.children):
            _last = n==(count-1)
            node._print_tree(_prefix, _last)
            
    def print_tree(self):
        print(self.name)
        for node in self.children[:-1]:
            node._print_tree(_last = False)
        self.children[-1]._print_tree(_last=True)
        
    
#     # Version 3
#     # 【不太能理解 递归的操作】
#     def print_tree(self, _prefix = '', _last = True):
# #         print(_prefix, '┗━━' if _last and self.name != 'root' else '┣━━', self.name, sep='')
#         print(_prefix, '┗━━' if _last else '┣━━', self.name, sep='')
#         _prefix += '   ' if _last else '┃  '
#         count = len(self.children)
#         for n, node in enumerate(self.children):
#             _last = n == (count - 1)
#             node.print_tree(_prefix, _last)

#     # Version 2
#     def print_tree(self, tier = 0):
#         prefix = "┃  " * (tier-1) + '┣━━' if tier>0 else ''  # tier 要改成 (tier-1)，不然会有棈一竖列多余的丨 
#         print(prefix + self.name)
#         if self.children:
#             for node in self.children:
#                 node.print_tree(tier+1)
    
#     # Version 1
#     def print_tree(self, tier=0):
#         prefix = '  ' * tier
#         print(prefix + self.name)
#         if self.children:  # 错误：按照 2020-01-21 的课程输入，发现不对，感觉应该改成 self，但是这里好几遍没有改进来
#             for node in self.children:  # 上面一行改了，但是这一行又是在后面一次改过来了
#                 node.print_tree(tier+1)  # 这是最后改成功的地方：这个 error 是最容易修改的，提示直接是 argu 多了，那就知道要把括号里面（node, tier+1）的 node 删掉，然后就出来结果了
                
#     下面是 2020-01-21 的课程的 Version 1：
#     def print_tree(root, tier=0):
#         prefix = ' ' * (tier - 1)
#         print(prefix + root.name)
#         if root.children and len(root.children):
#             for node in root.children:
#                 print_tree(node, tier + 1)
    

In [90]:
root = TreeNode('中国', {'name':'习近平', 'age':5000, 'capitals':'Beijing'})

In [91]:
a1 = TreeNode('北京', {'tag':'a1', 'name':'BeiJing', 'age':18, 'level':1})
a2 = TreeNode('上海', {'tag':'a2', 'name':'ShangHai', 'age':17, 'level':1})
a3 = TreeNode('广东', {'tag':'a3', 'name':'GuangZhou', 'age':10, 'level':1})
a4 = TreeNode('浙江', {'tag':'a4', 'name':'ShenZhen', 'age':8, 'level':1})
a5 = TreeNode('天津', {'tag':'a5'})
# a6 = TreeNode('黑龙江', {'tag':'a6'})
# a7 = TreeNode('吉林', {'tag':'a7'})
# a8 = TreeNode('辽宁', {'tag':'a8'})
# a9 = TreeNode('河北', {'tag':'a9'})
# a10 = TreeNode('河南', {'tag':'a10'})
# a11 = TreeNode('江西', {'tag':'a11'})
# a12 = TreeNode('山东', {'tag':'a12'})
# a13 = TreeNode('山西', {'tag':'a13'})
# a14 = TreeNode('陕西', {'tag':'a14'})
# a15 = TreeNode('湖北', {'tag':'a15'})
# a16 = TreeNode('湖南', {'tag':'a16'})
# a17 = TreeNode('甘肃', {'tag':'a17'})
# a18 = TreeNode('贵州', {'tag':'a18'})
# a19 = TreeNode('云南', {'tag':'a19'})
# a20 = TreeNode('广西', {'tag':'a20'})
# a21 = TreeNode('西藏', {'tag':'a21'})
# a22 = TreeNode('内蒙古', {'tag':'a22'})
# a23 = TreeNode('新疆', {'tag':'a23'})
# a24 = TreeNode('香港', {'tag':'a24'})
# a25 = TreeNode('澳门', {'tag':'a25'})
# a26 = TreeNode('台湾', {'tag':'a26'})
# a27 = TreeNode('重庆', {'tag':'a27'})
# a28 = TreeNode('福建', {'tag':'a28'})
# a29 = TreeNode('海南', {'tag':'a29'})
# a30 = TreeNode('四川', {'tag':'a30'})
# a31 = TreeNode('青海', {'tag':'a31'})
# a32 = TreeNode('宁夏', {'tag':'a32'})
# a33 = TreeNode('江苏', {'tag':'a33'})
# a34 = TreeNode('安徽', {'tag':'a34'})

# aJihe = []
# for i in range(34):
#     num = str(i+1)
#     ai = 'a'+ num
#     aJihe.append(ai)
    
# root.add_children(aJihe)
# root.add_children([a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15, a16, a17, a18, a19, a20, a21, a22, a23, a24, a25, a26, a27, a28, a29, a30, a31, a32, a33, a34])
root.add_children([a1, a2, a3, a4, a5])
# root.add_child(a1)
# root.add_child(a2)
# root.add_child(a3)
# root.add_child(a4)

In [92]:
shengfen = []
for child in root.children:
    shengfen.append(child.name)
shengfen_set = set(shengfen)
len(shengfen_set)

# 寻找哪个是重复的 code，添加‘海南’
# for i in range(34):
#     for j in range(i, 34):
#         if shengfen[i] == shengfen[j]:
#             print(i, j)

5

In [93]:
b11 = TreeNode('朝阳区', {'tag':'b11', 'name':'chaoyangqu', 'number':1000})
b12 = TreeNode('海淀区', {'tag':'b12', 'name':'haidingqu', 'number':1200})
b13 = TreeNode('天安门', {'tag':'b13', 'name':'tianojmenqu', 'number':0})

b21 = TreeNode('浦东区', {'tag':'b21', 'name':'hongqiao', 'number':900})
b22 = TreeNode('蒲西区', {'tag':'b22', 'name':'nanjinglu', 'number':800})
b23 = TreeNode('南京路', {'tag':'b23', 'name':'pudongqu', 'number':999})

b31 = TreeNode('广州', {'tag':'b31', 'name':'yuexiu', 'number':2000})
b32 = TreeNode('深圳', {'tag':'b32', 'name':'tianhe', 'number':1900})
b33 = TreeNode('东莞', {'tag':'b33', 'name':'liwan', 'number':1500})

b41 = TreeNode('杭州', {'tag':'b41', 'name':'huaweiqu', 'number':300})
b42 = TreeNode('宁波', {'tag':'b42', 'name':'tencent', 'number':251})
b43 = TreeNode('温州', {'tag':'b43', 'name':'dajiang', 'number':120})

c111 = TreeNode('小区1', {'tag':'c111', 'xcvh':'vhbhxn', 'fuxcvh':'xcm;'})
c112 = TreeNode('小区2', {'tag':'c112', 'hbqnvugr':'days', 'fuhbqnvugr':'xcys'})

In [94]:
a1.add_children([b11, b12, b13])
# a1.add_child(b11)
# a1.add_child(b12)
# a1.add_child(b13)
b11.add_children([c111, c112])
a2.add_children([b21, b22, b23])
a3.add_children([b31, b32, b33])
a4.add_children([b41, b42, b43])

In [96]:
print(root.get_tier('南京路'))

2


In [39]:
root.print_tree()

中国
┣━━北京
┃  ┣━━朝阳区
┃  ┃  ┣━━小区1
┃  ┃  ┗━━小区2
┃  ┣━━海淀区
┃  ┗━━天安门
┣━━上海
┃  ┣━━浦东区
┃  ┣━━蒲西区
┃  ┗━━南京路
┣━━广东
┃  ┣━━广州
┃  ┣━━深圳
┃  ┗━━东莞
┣━━浙江
┃  ┣━━杭州
┃  ┣━━宁波
┃  ┗━━温州
┗━━天津


In [9]:
root_descendants = root.get_descendants('北京')

北京
┣━━朝阳区
┃  ┣━━小区1
┃  ┗━━小区2
┣━━海淀区
┗━━天安门


In [11]:
# root.move(c111, b13)
# root.print_tree()

In [29]:
siblings = root.get_siblings('朝阳区')
for sibling in siblings:
    print(sibling.name)

海淀区
天安门


In [50]:
print(root.find('小区1').data['tag'])

c111


In [51]:
print(root.find('浙江').data['tag'])  # 第一层是 root，这个是第二层（省份）的一个
root.find('小区2')  # 这个是第三层的随便的内容
root.find('bala') # 这个是没有的，

a4


In [69]:
a35 = TreeNode('American', {})
root.add_child(a35)
root.print_tree()

中国
┣━━北京
┃  ┣━━朝阳区
┃  ┃  ┣━━小区1
┃  ┃  ┗━━小区2
┃  ┣━━海淀区
┃  ┗━━天安门
┣━━上海
┃  ┣━━浦东区
┃  ┣━━蒲西区
┃  ┗━━南京路
┣━━广东
┃  ┣━━广州
┃  ┣━━深圳
┃  ┗━━东莞
┣━━浙江
┃  ┣━━杭州
┃  ┣━━宁波
┃  ┗━━温州
┣━━天津
┣━━黑龙江
┣━━吉林
┣━━辽宁
┣━━河北
┣━━河南
┣━━江西
┣━━山东
┣━━山西
┣━━陕西
┣━━湖北
┣━━湖南
┣━━甘肃
┣━━贵州
┣━━云南
┣━━广西
┣━━西藏
┣━━内蒙古
┣━━新疆
┣━━香港
┣━━澳门
┣━━台湾
┣━━重庆
┣━━福建
┣━━海南
┣━━四川
┣━━青海
┣━━宁夏
┣━━江苏
┣━━安徽
┗━━American


In [71]:
root.add_child(a34)
root.remove(a35)
root.print_tree()

中国
┣━━北京
┃  ┣━━朝阳区
┃  ┃  ┣━━小区1
┃  ┃  ┗━━小区2
┃  ┣━━海淀区
┃  ┗━━天安门
┣━━上海
┃  ┣━━浦东区
┃  ┣━━蒲西区
┃  ┗━━南京路
┣━━广东
┃  ┣━━广州
┃  ┣━━深圳
┃  ┗━━东莞
┣━━浙江
┃  ┣━━杭州
┃  ┣━━宁波
┃  ┗━━温州
┣━━天津
┣━━黑龙江
┣━━吉林
┣━━辽宁
┣━━河北
┣━━河南
┣━━江西
┣━━山东
┣━━山西
┣━━陕西
┣━━湖北
┣━━湖南
┣━━甘肃
┣━━贵州
┣━━云南
┣━━广西
┣━━西藏
┣━━内蒙古
┣━━新疆
┣━━香港
┣━━澳门
┣━━台湾
┣━━重庆
┣━━福建
┣━━海南
┣━━四川
┣━━青海
┣━━宁夏
┣━━江苏
┗━━安徽


### 理解-sxuya
1. 验证两步：parent 和 children
2. 每个验证都要先确认是不是一致的数据结构；
3. 如果是，则进行关系的添加。（parent 节点只有一个，而 children 很多，所以使用 list 来创建）
4. 无论是添加 parent 还是添加 children，其实都是两个建立关系，一定是一个是 parent、一个是 children，所以，添加 parent 还是添加 children，其实本质的方法是一样的。
5. add_child 这个方法的名字有点让初学者费解，更好的理解应该就是「建立 parent-child 联系」。

我们设计的核心数据结构是表示树节点的自定义类型 TreeNode，这个类型的对象有四个实例变量（属性）：
* *name*：节点的名字，最好能唯一标识出一个节点
* *data*：节点相关的任何数据，可以是任何数据类型
* *parent*：节点的父节点，如果没有父节点就是 None
* *children*：节点所有子节点组成的一个列表

这里面 *parent* 和 *children* 里的元素都必须是 TreeNode 类型的对象，我们在处理这两个属性时要先确认这一点，在上面的代码中我们用 Python 的 `assert` 语句和 `isinstance()` 函数来实现：
* `assert` 关键字后面的表达式必须返回 True，否则程序将抛出 `AssertionError` 异常后终止；
* `isinstance(obj, type)` 之前我们就介绍过，它接受两个参数，第一个参数是一个对象，而第二个参数是一个类型，函数判断第一个参数是不是第二个参数指明的类型，如果是返回 `True`，否则返回 `False`。

如上定义的 `TreeNode` 类，它实例化出来的对象 `node` 具备如下的能力：
* 很容易取得其父节点 *parent*；
* 很容易取得其所有子节点 *children*；
* 已经实现了增加子节点的操作。

最有意思的是，TreeNode 类型实际上也代表了树本身，因为一个节点加上它所有子节点，本来就是一棵树嘛！

在这个类型定义的基础上，我们可以实现前一节列出的所有操作，你可以把这作为练习动手试一试。

> 在实际项目中，我们并不需要定义树的数据结构，因为有优秀的第三方实现可用，比如 Python 非常棒的第三方库 [anytree](https://anytree.readthedocs.io/en/latest/#)，你可以试试，看和你自己实现的有什么区别。

## 小结

这一章介绍非常常见的树形逻辑怎么实现。没有很多代码，主要讲的是思维方法，请你读完一遍之后尝试自己从头思考和建立一个基本的树的数据结构，如果遇到问题就再读一遍。